# Class Definitions

In [5]:
class Tree:
    def __init__(self,data=None):
        self.root = data
        
    def pr(self):
        self.root.pr()
    def __repr__(self):
        return self.root.__repr__()
        
    def set_root(self, data):
        self.root = data
    
    def get_root(self):
        return self.root
    
    def addPath(self,ll):
        if self.root == None:
            #print(ll[0], 'created')
            self.root = Nodes(ll[0],[])
        #print('will add', ll[1:])
        self.root.addPath(ll[1:])
        
class Nodes:
    def __init__(self,val = None, childs = []):
        self.val = val
        self.childs = childs
        
        
    def pr(self):
        print('my value:',self.val)
        print('')
        #print('my childs:', self.childs)
        for i in range(len(self.childs)):
            self.childs[i].pr()
            
     # prints data
    def __repr__(self):
        
        #for i in self.childs:
            #print(i)
        return str(self.val)
    
   
            
    # returns the index of given data in childs else -1
    def isInChild(self, x):
        
        for i in range(len(self.childs)):
            if self.childs[i].val == x:
                
                return i
               
        return -1
            
    # if x data is not in childs adds it to the child        
    def addToChilds(self, x):
        #print('addtochild called on', self.val, self.childs)
        index = self.isInChild(x)
        #print(x, 'exist @', index)
        if index == -1:
            #print(x, 'added to child')
            nodex = Nodes(x,[])
            self.childs.append(nodex)
            return nodex
        else:
            return self.childs[index]
        
    # gets a path as a list adds to the Tree    
    def addPath(self, path):
        #print('addpath called on', self.val)
        if len(path) == 0:
            return
        else:
            self.addToChilds(path[0])
            index = self.isInChild(path[0])
            #print('confirm index added @', index, self.childs)
            dummy = self.childs[index]
            #print('calling addpath', path[1:], 'on', dummy)
            dummy.addPath(path[1:])
            return


        
        
class NodeLabels():
    def __init__(self):
        self.pairs = {}
        self.reversepairs = {}
        self.cur_label = 0
    def addNode(self,val):
        if not val in self.pairs.keys():
            self.reversepairs.update({str(self.cur_label) : str(val)})
            self.pairs.update({val : str(self.cur_label)})
            self.cur_label = self.cur_label + 1
            
    def getVal(self, key):
        return int(self.pairs[key])
    
    def get(self):
        return self.pairs
    
    def get_reverse(self):
        return self.reversepairs
    


# Test tree

In [6]:
e = Nodes('e')
f = Nodes('f')
g = Nodes('g')
efg = [e,f,g]
b = Nodes('b', efg)

d = Nodes('d')
c = Nodes('c',[Nodes('z',[Nodes('k')])])

a = Nodes('a', [b,c,d])
test = Tree(a)



In [7]:
test.pr()

my value: a

my value: b

my value: e

my value: f

my value: g

my value: c

my value: z

my value: k

my value: d



In [8]:
print(test)

a


# Tree2JSON

In [9]:
def getedges(t):
    edges = []
    curNode = t.root
    dic = NodeLabels()
    return accedges(curNode, None, edges, dic)
    
def accedges(node, parent, edges, dic):
    if parent == None:
        for i in range(len(node.childs)):
            accedges(node.childs[i],node.val, edges, dic)
    else:
        dic.addNode(parent)
        dic.addNode(node.val)
        edges.append([dic.getVal(parent), dic.getVal(node.val)])
        for i in range(len(node.childs)):
            accedges(node.childs[i],node.val, edges, dic)
    return {'edges': edges, 'labels':dic.get(), 'r_labels':dic.get_reverse()}
    

# Example

In [25]:
getedges(test)

{'edges': [[0, 1], [1, 2], [1, 3], [1, 4], [0, 5], [5, 6], [6, 7], [0, 8]],
 'labels': {'a': '0',
  'b': '1',
  'c': '5',
  'd': '8',
  'e': '2',
  'f': '3',
  'g': '4',
  'k': '7',
  'z': '6'},
 'r_labels': {'0': 'a',
  '1': 'b',
  '2': 'e',
  '3': 'f',
  '4': 'g',
  '5': 'c',
  '6': 'z',
  '7': 'k',
  '8': 'd'}}

# JSON2Tree

In [11]:
def recontructTree(json_dict):
    rough_tree = jsontoTree(json_dict)
    cur_tree = Tree()
    root_key = findrootkey(rough_tree)
    print(root_key)
    cur_tree.set_root(reconstNode(rough_tree,root_key))
    return cur_tree

def findrootkey(rough_tree):
    cur_root = None
    for i in rough_tree.keys():
        if not keyinvalues(i,rough_tree.values()):
            cur_root = i
            return cur_root
    return cur_root
        
        
def keyinvalues(key, ll):
    for i in ll:
        if key in i:
            return True
    return False

def reconstNode(rough_tree, key):
    childs = []
    if not key in rough_tree.keys():
        return Nodes(key,childs)
    
    c = rough_tree[key]
    for i in c:
        childs.append(reconstNode(rough_tree, i))
    
    return Nodes(key, childs)
            

In [20]:
def jsontoTree(json_dict):
    edges = json_dict['edges']
    labels = json_dict['labels']
    edges_dict = {}
    for i in edges:
        if labels[str(i[0])] in edges_dict.keys():
            edges_dict[labels[str(i[0])]].append(labels[str(i[1])])
        else:
            edges_dict.update({labels[str(i[0])] : [labels[str(i[1])]]})
    return edges_dict

In [24]:
jsontoTree({"target": 1, "labels": {"0": "1", "1": "2", "2": "3", "3": "3", "4": "1", "5": "1", "6": "3"}, "edges": [[0, 1], [1, 2], [2, 3], [2, 6], [3, 5], [3, 6], [4, 6]]})

{'1': ['2', '3'], '2': ['3'], '3': ['3', '3', '1', '3']}

In [26]:
A={"r_labels": {"txtvsbin.txt": "46", "zlibstat.vcxproj": "159", "unzip.c": "227", "match.S": "104", "DotZLib.sln": "187", "gvmat64.asm": "123", "zutil.h": "41", "CodecBase.cs": "195", "TODO": "137", "DotZLib.chm": "189", "zlib.adb": "133", "adler32.c": "19", "infback.c": "39", "zlibvc.sln": "156", "minizip.vcxproj": "163", "gzlog.h": "73", "configure.ac": "219", "Makefile.in": "31", "LICENSE_1_0.txt": "190", "zlib": "0", "vc9": "148", "inflate9.h": "180", "README.686": "105", "vc12": "166", "zran.c": "78", "match686.asm": "100", "example.pas": "144", "iowin32.c": "223", "inffas86.c": "176", "Inflater.cs": "201", "inftrees.h": "23", "amd64-match.S": "141", "zstream_test.cpp": "208", "ChangeLog": "32", "test.txt": "205", "infcover.c": "91", "configure": "37", "GZipStream.cs": "198", "deflate.c": "40", "test.cc": "138", "gzread.c": "9", "fitblk.c": "80", "MiniZip64_info.txt": "229", "zconf.h.in": "22", "Makefile.dj2": "94", "testzlib.c": "112", "zlib.rc": "150", "buffer_demo.adb": "132", "zlib-thin.adb": "135", "amd64": "140", "inftrees.c": "7", "zip.h": "231", "zlibd32.mak": "143", "bld_ml64.bat": "121", "read.adb": "131", "gzguts.h": "20", "inftree9.h": "183", "inffast.h": "35", "zlibvc.def": "153", "Makefile.os2": "89", "qnx": "234", "iostream3": "136", "testzlib": "111", "README.examples": "74", "enough.c": "82", "zip.c": "220", "inflate86": "174", "ioapi.c": "222", "zlib.pc.in": "6", "puff.c": "110", "VisualC.txt": "61", "AssemblyInfo.cs": "193", "bld_ml32.bat": "99", "gzclose.c": "13", "zlib1.rc": "60", "crc32.h": "10", "gzjoin.c": "79", "zlib.pc.cmakein": "3", "rfc1950.txt": "47", "zstream.h": "209", "gzappend.c": "75", "minigzip.c": "92", "ChecksumImpl.cs": "194", "zlib2ansi": "18", "ZLibConst.pas": "211", "crc32.c": "26", "zconf.h": "12", "miniunz.vcproj": "151", "zlibpas.pas": "145", "inffasx64.asm": "122", "treebuild.xml": "24", "DotZLib.build": "188", "bndsrc": "65", "zeros.raw": "108", "os2": "88", "zlib.def": "56", "make_vms.com": "17", "minizip.vcxproj.filters": "173", "DLL_FAQ.txt": "62", "watcom_f.mak": "69", "Makefile.riscos": "85", "zlib.gpr": "125", "INDEX": "4", "ioapi.h": "230", "Makefile.emx": "87", "mztools.c": "221", "zlib.ads": "134", "zutil.c": "11", "untgz": "114", "inffixed.h": "2", "inffas32.asm": "102", "descrip.mms": "84", "gzlog.c": "72", "gcc_gvmat64": "177", "zlibvc.vcxproj": "161", "asm686": "103", "os400": "63", "puff": "106", "rfc1952.txt": "49", "gzlib.c": "29", "untgz.c": "115", "watcom_l.mak": "70", "zlib.map": "1", "vstudio": "147", "infback9": "179", "testzlib.vcproj": "149", "rfc1951.txt": "50", "crypt.h": "216", "README400": "67", "iostream2": "207", "gzwrite.c": "42", "DotZLib.csproj": "200", "exampl": "71", "minizip.1": "217", "testzlib.vcxproj.filters": "171", "Makefile.tc": "95", "vc11": "165", "inffix9.h": "185", "CircularBuffer.cs": "192", "minizip.pc.in": "228", "msd": "93", "compress.c": "44", "zlib.3.pdf": "28", "Makefile.pup": "52", "trees.c": "34", "zlib.h": "5", "inffast.S": "175", "miniunz.c": "214", "README": "21", "MiniZip64_Changes.txt": "215", "zlib.inc": "64", "masmx86": "98", "minizip.c": "225", "dotzlib": "186", "infback9.c": "181", "UnitTests.cs": "199", "inflate.h": "43", "README-WIN32.txt": "59", "vc10": "167", "test.pk": "206", "miniunz.vcxproj": "164", "miniz": "213", "zlib.3": "25", "zlib_how.html": "81", "testzlibdll.vcproj": "152", "Deflater.cs": "197", "miniunzip.1": "218", "miniunz.vcxproj.filters": "168", "puff.h": "107", "mztools.h": "232", "vc14": "158", "zlibvc.vcxproj.filters": "169", "gun.c": "77", "Makefile.sas": "53", "testzlibdll.vcxproj": "160", "trees.h": "8", "blast.c": "204", "algorithm.txt": "48", "deflate.h": "36", "contrib": "96", "Makefile.gcc": "57", "w": "68", "inflate.c": "16", "iowin32.h": "226", "Makefile.bor": "55", "testzlib.txt": "113", "make.sh": "66", "visual-basic.txt": "86", "Makefile.msc": "58", "package.qpg": "235", "readme.txt": "101", "inffas8664.c": "120", "ad": "124", "zlibstat.vcxproj.filters": "172", "FAQ": "33", "zconf.h.cmakein": "30", "d": "45", "bl": "202", "inftree9.c": "184", "gvmat64.S": "178", "infback9.h": "182", "win32": "54", "zfstream.cc": "139", "pufftest.c": "109", "old": "83", "DotZLib": "191", "example.c": "90", "nintendod": "51", "Makefile": "27", "zlibstat.vcproj": "154", "vstud": "146", "ZLib.pas": "212", "zfstream.cpp": "118", "zlib-thin.ads": "128", "zfstream.h": "117", "uncompr.c": "15", "testzlibdll.vcxproj.filters": "170", "zlib-streams.ads": "127", "testzlib.vcxproj": "162", "CMakeLists.txt": "38", "test.cpp": "116", "minizip.vcproj": "157", "zpipe.c": "76", "README.contrib": "97", "test.adb": "126", "del": "210", "zlibvc.vcproj": "155", "pascal": "142", "inffast.c": "14", "blast.h": "203", "zlib-streams.adb": "130", "mtest.adb": "129", "unzip.h": "224", "masmx64": "119", "Makefile.am": "233", "DotZLib.cs": "196"}, "labels": {"168": "miniunz.vcxproj.filters", "108": "zeros.raw", "206": "test.pk", "14": "inffast.c", "161": "zlibvc.vcxproj", "210": "del", "207": "iostream2", "188": "DotZLib.build", "213": "miniz", "187": "DotZLib.sln", "105": "README.686", "196": "DotZLib.cs", "76": "zpipe.c", "125": "zlib.gpr", "18": "zlib2ansi", "198": "GZipStream.cs", "91": "infcover.c", "39": "infback.c", "200": "DotZLib.csproj", "201": "Inflater.cs", "170": "testzlibdll.vcxproj.filters", "134": "zlib.ads", "164": "miniunz.vcxproj", "141": "amd64-match.S", "169": "zlibvc.vcxproj.filters", "222": "ioapi.c", "177": "gcc_gvmat64", "59": "README-WIN32.txt", "45": "d", "5": "zlib.h", "233": "Makefile.am", "208": "zstream_test.cpp", "152": "testzlibdll.vcproj", "77": "gun.c", "172": "zlibstat.vcxproj.filters", "163": "minizip.vcxproj", "71": "exampl", "144": "example.pas", "189": "DotZLib.chm", "75": "gzappend.c", "179": "infback9", "218": "miniunzip.1", "197": "Deflater.cs", "32": "ChangeLog", "167": "vc10", "121": "bld_ml64.bat", "35": "inffast.h", "65": "bndsrc", "130": "zlib-streams.adb", "70": "watcom_l.mak", "135": "zlib-thin.adb", "226": "iowin32.h", "7": "inftrees.c", "183": "inftree9.h", "84": "descrip.mms", "85": "Makefile.riscos", "69": "watcom_f.mak", "72": "gzlog.c", "106": "puff", "38": "CMakeLists.txt", "20": "gzguts.h", "148": "vc9", "181": "infback9.c", "96": "contrib", "199": "UnitTests.cs", "13": "gzclose.c", "186": "dotzlib", "182": "infback9.h", "92": "minigzip.c", "19": "adler32.c", "6": "zlib.pc.in", "1": "zlib.map", "215": "MiniZip64_Changes.txt", "37": "configure", "190": "LICENSE_1_0.txt", "122": "inffasx64.asm", "16": "inflate.c", "165": "vc11", "87": "Makefile.emx", "132": "buffer_demo.adb", "8": "trees.h", "23": "inftrees.h", "88": "os2", "191": "DotZLib", "149": "testzlib.vcproj", "95": "Makefile.tc", "78": "zran.c", "153": "zlibvc.def", "101": "readme.txt", "205": "test.txt", "24": "treebuild.xml", "28": "zlib.3.pdf", "124": "ad", "216": "crypt.h", "112": "testzlib.c", "120": "inffas8664.c", "60": "zlib1.rc", "235": "package.qpg", "229": "MiniZip64_info.txt", "180": "inflate9.h", "221": "mztools.c", "160": "testzlibdll.vcxproj", "228": "minizip.pc.in", "178": "gvmat64.S", "114": "untgz", "11": "zutil.c", "157": "minizip.vcproj", "94": "Makefile.dj2", "27": "Makefile", "51": "nintendod", "0": "zlib", "44": "compress.c", "166": "vc12", "202": "bl", "54": "win32", "10": "crc32.h", "22": "zconf.h.in", "56": "zlib.def", "231": "zip.h", "145": "zlibpas.pas", "127": "zlib-streams.ads", "162": "testzlib.vcxproj", "136": "iostream3", "64": "zlib.inc", "34": "trees.c", "230": "ioapi.h", "209": "zstream.h", "175": "inffast.S", "82": "enough.c", "55": "Makefile.bor", "138": "test.cc", "223": "iowin32.c", "90": "example.c", "62": "DLL_FAQ.txt", "99": "bld_ml32.bat", "139": "zfstream.cc", "83": "old", "17": "make_vms.com", "25": "zlib.3", "46": "txtvsbin.txt", "79": "gzjoin.c", "203": "blast.h", "103": "asm686", "53": "Makefile.sas", "74": "README.examples", "66": "make.sh", "49": "rfc1952.txt", "159": "zlibstat.vcxproj", "150": "zlib.rc", "30": "zconf.h.cmakein", "155": "zlibvc.vcproj", "131": "read.adb", "133": "zlib.adb", "47": "rfc1950.txt", "173": "minizip.vcxproj.filters", "113": "testzlib.txt", "80": "fitblk.c", "142": "pascal", "212": "ZLib.pas", "119": "masmx64", "194": "ChecksumImpl.cs", "117": "zfstream.h", "57": "Makefile.gcc", "100": "match686.asm", "123": "gvmat64.asm", "41": "zutil.h", "227": "unzip.c", "225": "minizip.c", "115": "untgz.c", "52": "Makefile.pup", "43": "inflate.h", "9": "gzread.c", "58": "Makefile.msc", "97": "README.contrib", "42": "gzwrite.c", "109": "pufftest.c", "68": "w", "63": "os400", "102": "inffas32.asm", "137": "TODO", "220": "zip.c", "21": "README", "12": "zconf.h", "111": "testzlib", "176": "inffas86.c", "174": "inflate86", "214": "miniunz.c", "217": "minizip.1", "31": "Makefile.in", "67": "README400", "224": "unzip.h", "185": "inffix9.h", "36": "deflate.h", "204": "blast.c", "73": "gzlog.h", "151": "miniunz.vcproj", "61": "VisualC.txt", "48": "algorithm.txt", "110": "puff.c", "140": "amd64", "232": "mztools.h", "211": "ZLibConst.pas", "146": "vstud", "3": "zlib.pc.cmakein", "89": "Makefile.os2", "171": "testzlib.vcxproj.filters", "192": "CircularBuffer.cs", "26": "crc32.c", "81": "zlib_how.html", "50": "rfc1951.txt", "126": "test.adb", "15": "uncompr.c", "129": "mtest.adb", "143": "zlibd32.mak", "29": "gzlib.c", "40": "deflate.c", "147": "vstudio", "195": "CodecBase.cs", "4": "INDEX", "219": "configure.ac", "193": "AssemblyInfo.cs", "118": "zfstream.cpp", "104": "match.S", "107": "puff.h", "98": "masmx86", "116": "test.cpp", "156": "zlibvc.sln", "93": "msd", "154": "zlibstat.vcproj", "33": "FAQ", "86": "visual-basic.txt", "128": "zlib-thin.ads", "158": "vc14", "184": "inftree9.c", "234": "qnx", "2": "inffixed.h"}, "edges": [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10], [0, 11], [0, 12], [0, 13], [0, 14], [0, 15], [0, 16], [0, 17], [0, 18], [0, 19], [0, 20], [0, 21], [0, 22], [0, 23], [0, 24], [0, 25], [0, 26], [0, 27], [0, 28], [0, 29], [0, 30], [0, 31], [0, 32], [0, 33], [0, 34], [0, 35], [0, 36], [0, 37], [0, 38], [0, 39], [0, 40], [0, 41], [0, 42], [0, 43], [0, 44], [0, 45], [45, 46], [45, 47], [45, 48], [45, 49], [45, 50], [0, 51], [51, 21], [51, 27], [0, 52], [0, 53], [0, 54], [54, 55], [54, 56], [54, 57], [54, 58], [54, 59], [54, 60], [54, 61], [54, 62], [0, 63], [63, 64], [63, 65], [63, 66], [63, 67], [0, 68], [68, 69], [68, 70], [0, 71], [71, 72], [71, 73], [71, 74], [71, 75], [71, 76], [71, 77], [71, 78], [71, 79], [71, 80], [71, 81], [71, 82], [0, 83], [83, 84], [83, 21], [83, 85], [83, 86], [83, 87], [83, 88], [88, 56], [88, 89], [0, 90], [0, 91], [0, 92], [0, 93], [93, 55], [93, 94], [93, 58], [93, 95], [93, 87], [0, 96], [96, 97], [96, 98], [98, 99], [98, 100], [98, 101], [98, 102], [96, 103], [103, 104], [103, 105], [96, 106], [106, 107], [106, 21], [106, 108], [106, 27], [106, 109], [106, 110], [96, 111], [111, 112], [111, 113], [96, 114], [114, 115], [114, 58], [114, 27], [96, 116], [96, 117], [96, 118], [96, 119], [119, 120], [119, 121], [119, 101], [119, 122], [119, 123], [96, 124], [124, 125], [124, 126], [124, 127], [124, 101], [124, 128], [124, 129], [124, 130], [124, 131], [124, 132], [124, 133], [124, 134], [124, 135], [96, 136], [136, 137], [136, 138], [136, 117], [136, 21], [136, 139], [96, 140], [140, 141], [96, 142], [142, 143], [142, 101], [142, 144], [142, 145], [96, 146], [146, 101], [96, 147], [147, 148], [148, 149], [148, 150], [148, 151], [148, 152], [148, 153], [148, 154], [148, 155], [148, 156], [148, 157], [147, 158], [158, 159], [158, 160], [158, 150], [158, 161], [158, 162], [158, 153], [158, 163], [158, 164], [158, 156], [147, 165], [165, 159], [165, 160], [165, 150], [165, 161], [165, 162], [165, 153], [165, 163], [165, 164], [165, 156], [147, 166], [166, 159], [166, 160], [166, 150], [166, 161], [166, 162], [166, 153], [166, 163], [166, 164], [166, 156], [147, 167], [167, 168], [167, 169], [167, 170], [167, 159], [167, 160], [167, 150], [167, 161], [167, 162], [167, 171], [167, 153], [167, 163], [167, 164], [167, 172], [167, 156], [167, 173], [96, 174], [174, 175], [174, 176], [96, 177], [177, 178], [96, 179], [179, 180], [179, 181], [179, 21], [179, 182], [179, 183], [179, 184], [179, 185], [96, 186], [186, 187], [186, 101], [186, 188], [186, 189], [186, 190], [186, 191], [191, 192], [191, 193], [191, 194], [191, 195], [191, 196], [191, 197], [191, 198], [191, 199], [191, 200], [191, 201], [96, 202], [202, 203], [202, 204], [202, 21], [202, 27], [202, 205], [202, 206], [96, 207], [207, 208], [207, 209], [96, 210], [210, 143], [210, 211], [210, 101], [210, 212], [96, 213], [213, 214], [213, 215], [213, 216], [213, 217], [213, 218], [213, 219], [213, 220], [213, 221], [213, 17], [213, 222], [213, 223], [213, 27], [213, 224], [213, 225], [213, 226], [213, 227], [213, 228], [213, 229], [213, 230], [213, 231], [213, 232], [213, 233], [0, 234], [234, 235]], "target": 2}


In [29]:
recontructTree(A).pr()

zlib
my value: zlib

my value: zlib.map

my value: inffixed.h

my value: zlib.pc.cmakein

my value: INDEX

my value: zlib.h

my value: zlib.pc.in

my value: inftrees.c

my value: trees.h

my value: gzread.c

my value: crc32.h

my value: zutil.c

my value: zconf.h

my value: gzclose.c

my value: inffast.c

my value: uncompr.c

my value: inflate.c

my value: make_vms.com

my value: zlib2ansi

my value: adler32.c

my value: gzguts.h

my value: README

my value: zconf.h.in

my value: inftrees.h

my value: treebuild.xml

my value: zlib.3

my value: crc32.c

my value: Makefile

my value: zlib.3.pdf

my value: gzlib.c

my value: zconf.h.cmakein

my value: Makefile.in

my value: ChangeLog

my value: FAQ

my value: trees.c

my value: inffast.h

my value: deflate.h

my value: configure

my value: CMakeLists.txt

my value: infback.c

my value: deflate.c

my value: zutil.h

my value: gzwrite.c

my value: inflate.h

my value: compress.c

my value: d

my value: txtvsbin.txt

my value: rfc1950.txt

m

# Example

In [16]:
recontructTree({'edges': [[0, 1], [1, 2], [1, 3], [1, 4], [0, 5], [5, 6], [6, 7], [0, 8]],
 'r_labels': {'a': '0',
  'b': '1',
  'c': '5',
  'd': '8',
  'e': '2',
  'f': '3',
  'g': '4',
  'k': '7',
  'z': '6'},
 'labels': {'0': 'a',
  '1': 'b',
  '2': 'e',
  '3': 'f',
  '4': 'g',
  '5': 'c',
  '6': 'z',
  '7': 'k',
  '8': 'd'}}).pr()

a
my value: a

my value: b

my value: e

my value: f

my value: g

my value: c

my value: z

my value: k

my value: d



In [43]:
def subtrees(tree):
    st = []
    subtrees_node(tree.root, st)
    return st
    
def subtrees_node(node, st):
    if node == None:
        return
    if node.childs == []:
        return
    for i in node.childs:
        subtrees_node(i, st)
    st.append(Tree(node))

In [44]:
zaa = subtrees(recontructTree({'edges': [[0, 1], [1, 2], [1, 3], [1, 4], [0, 5], [5, 6], [6, 7], [0, 8]],
 'labels': {'a': '0',
  'b': '1',
  'c': '5',
  'd': '8',
  'e': '2',
  'f': '3',
  'g': '4',
  'k': '7',
  'z': '6'},
 'r_labels': {'0': 'a',
  '1': 'b',
  '2': 'e',
  '3': 'f',
  '4': 'g',
  '5': 'c',
  '6': 'z',
  '7': 'k',
  '8': 'd'}}))

a


In [45]:
def printSubTrees(Tr):
    for i in Tr:
        print('NEW TREE')
        i.pr()
        print(getedges(i))
        print('============')
    

In [46]:
printSubTrees(zaa)

NEW TREE
my value: b

my value: e

my value: f

my value: g

{'r_labels': {'1': 'e', '0': 'b', '3': 'g', '2': 'f'}, 'labels': {'e': '1', 'f': '2', 'b': '0', 'g': '3'}, 'edges': [[0, 1], [0, 2], [0, 3]]}
NEW TREE
my value: z

my value: k

{'r_labels': {'1': 'k', '0': 'z'}, 'labels': {'k': '1', 'z': '0'}, 'edges': [[0, 1]]}
NEW TREE
my value: c

my value: z

my value: k

{'r_labels': {'1': 'z', '0': 'c', '2': 'k'}, 'labels': {'k': '2', 'z': '1', 'c': '0'}, 'edges': [[0, 1], [1, 2]]}
NEW TREE
my value: a

my value: b

my value: e

my value: f

my value: g

my value: c

my value: z

my value: k

my value: d

{'r_labels': {'4': 'g', '7': 'k', '5': 'c', '3': 'f', '6': 'z', '1': 'b', '0': 'a', '2': 'e', '8': 'd'}, 'labels': {'z': '6', 'a': '0', 'k': '7', 'e': '2', 'f': '3', 'b': '1', 'c': '5', 'd': '8', 'g': '4'}, 'edges': [[0, 1], [1, 2], [1, 3], [1, 4], [0, 5], [5, 6], [6, 7], [0, 8]]}


# Creating Training/Testing/Prediction Data

In [47]:
# Creates an array of objects
def pathTolistform(path):
          
    return path.split('/')

# Dot product
def completePaths(path, lof):
    comp = []
    for i in lof:
        name = i
        if name == None:
            continue
        cp = path + '/' + name
        
        comp.append(pathTolistform(cp))
   
    return comp

# Crawls the given path and extracts the omit path
def crawl(svn_root_path, omit=""):
    print("CRAWLING STARTED")
    print('')
    import os
    from os.path import join, getsize
    paths = []
    txtfiles = []
    for root, dirs, files in os.walk(svn_root_path):
        if files != []:
            paths.append(root.strip(omit))
            txtfiles.append(files)
    print("CRAWLING ENDEDD")
    print('')
    #print("Number of paths found: ",len(paths))
    print('')
    return [paths,txtfiles]

# Merges two dictionaries
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

# Dot product on all elements
def completeallPaths(paths, llof):
    
    all_l = []
    del_c ={}  
    for i in range(len(paths)):
        partial = completePaths(paths[i], llof[i])
        cur_key = partial[0][1]
        if cur_key in del_c.keys():
            del_c[cur_key] = del_c[cur_key] + len(partial)
        else:
            del_c.update({cur_key : len(partial)})
        all_l = all_l + partial
    print('found',del_c)
    print(' ')
    return all_l

# Adds paths to a tree
def lltoTree(path, retval = Tree(None)):
    for i in range(len(path)):
        #print(path[i])
        retval.addPath(path[i])
    return retval
    

In [48]:
# Given a list of root trees from the root directory extracts the elements as trees
def getAllTrees(lrootTree):
    allTrees = []
    for rootTree in lrootTree:
        for i in rootTree.root.childs:
            allTrees.append(Tree(i))
    return allTrees

In [49]:
# Gets trees generates json format data
# If test_train true will generate labeling as weel for testing and training
def getAlledges(allTrees, test_train = True):
    alledges = []
    target_dic = {}
    reverse_target_dic ={}
    cur_target = 0
    for i in allTrees:
        edges = getedges(i)
        if test_train:
            target = i.root.val
            if not target in reverse_target_dic.keys():
                reverse_target_dic.update({target:cur_target})
                target_dic.update({cur_target:target})
                cur_target = cur_target + 1
            edges.update({'target':reverse_target_dic[target]})
        alledges.append(edges)
    return alledges, target_dic

In [50]:
# Deletes 5% to 20% of given list randomly
def deleteRandomPaths(allPaths):
    import random
    mylist = allPaths
    n_elements = random.randint(len(mylist)*5//100, len(mylist)*20//100)
    print('deleting',n_elements,'out of',len(mylist))
    print(' ')
    random.shuffle(mylist)
    del_c ={}
    for i in range(0,n_elements):
        cur_key = mylist[0][1]
        if cur_key in del_c.keys():
            del_c[cur_key] = del_c[cur_key] + 1
        else:
            del_c.update({cur_key : 0})
        mylist.pop
        random.shuffle(mylist)
    print('deleted:', del_c)
    print(' ')
    return mylist    
    

In [51]:
# Takes the root directory of samples, and unneccessary preceding apth
# If test_train true class mappings and labeling will be added
# If createRandomData True randomsamples(100) new random trees will be generated
def createData(path, omitpath, test_train = True, createRandomData = True, Randomsamples = 100, realsamples = 1):
    import copy
    crawl_Res = crawl(path, omitpath)
    allPaths = completeallPaths(crawl_Res[0],crawl_Res[1]) # remove random paths to create test data
    rootTree = lltoTree(allPaths) # one tree
    lrootTree = []
    lrootTree.append(rootTree)
    for i in range(realsamples-1):
        print('creating deep copy of trees')
        print(' ')
        lrootTree.append(copy.deepcopy(rootTree))
        
    if createRandomData:
        print('creating randomized trees')
        print(' ')
        for i in range(Randomsamples):
            lrootTree.append(lltoTree(deleteRandomPaths(allPaths)))
        
        
    allTrees = getAllTrees(lrootTree) #list of trees
    
    allJSON = getAlledges(allTrees, test_train) #list of json, class mapping
    print('Class Mappings: ', allJSON[1])
    print(' ')
    print('Number of samples: ', len(allJSON[0]))
    return allJSON

# Examples

In [52]:
ret = createData('/home/eguray/Desktop/test_oss/','/home/eguray/Desktop/', False, False)

CRAWLING STARTED

CRAWLING ENDEDD


found {'curl': 3077, 'zlib': 253, 'openssl': 3336}
 
Class Mappings:  {}
 
Number of samples:  3


In [53]:
ret = createData('/home/eguray/Desktop/test_oss/','/home/eguray/Desktop/', True, False, 2)

CRAWLING STARTED

CRAWLING ENDEDD


found {'curl': 3077, 'zlib': 253, 'openssl': 3336}
 
Class Mappings:  {0: 'curl', 1: 'openssl', 2: 'zlib'}
 
Number of samples:  3


In [54]:
ret = createData('/home/eguray/Desktop/test_oss/','/home/eguray/Desktop/', True, True, 4,10)

CRAWLING STARTED

CRAWLING ENDEDD


found {'curl': 3077, 'zlib': 253, 'openssl': 3336}
 
creating deep copy of trees
 
creating deep copy of trees
 
creating deep copy of trees
 
creating deep copy of trees
 
creating deep copy of trees
 
creating deep copy of trees
 
creating deep copy of trees
 
creating deep copy of trees
 
creating deep copy of trees
 
creating randomized trees
 
deleting 444 out of 6666
 
deleted: {'curl': 216, 'zlib': 17, 'openssl': 208}
 
deleting 946 out of 6666
 
deleted: {'curl': 450, 'zlib': 33, 'openssl': 460}
 
deleting 795 out of 6666
 
deleted: {'curl': 378, 'zlib': 29, 'openssl': 385}
 
deleting 621 out of 6666
 
deleted: {'curl': 289, 'zlib': 23, 'openssl': 306}
 
Class Mappings:  {0: 'curl', 1: 'openssl', 2: 'zlib'}
 
Number of samples:  42


In [55]:
ret[1]


{0: 'curl', 1: 'openssl', 2: 'zlib'}

In [180]:
def saveAsJSON(scan_path = '/home/eguray/Desktop/test_oss/', scan_omit_path = '/home/eguray/Desktop/', test_split = 2, train_split = 8, save_path = None, test_train = True, createRandomData = True, Randomsamples = 4, realsamples = 10):
    raw_dic_data, map_data =  createData(scan_path, scan_omit_path, test_train, createRandomData, Randomsamples,realsamples)
    

In [182]:
saveAsJSON()

CRAWLING STARTED

CRAWLING ENDEDD


found {'curl': 3077, 'openssl': 3336, 'zlib': 253}
 
creating deep copy of trees
 
creating deep copy of trees
 
creating deep copy of trees
 
creating deep copy of trees
 
creating deep copy of trees
 
creating deep copy of trees
 
creating deep copy of trees
 
creating deep copy of trees
 
creating deep copy of trees
 
creating randomized trees
 
deleting 1231 out of 6666
 
deleted: {'openssl': 636, 'curl': 538, 'zlib': 54}
 
deleting 764 out of 6666
 
deleted: {'openssl': 384, 'curl': 344, 'zlib': 33}
 
deleting 1236 out of 6666
 
deleted: {'curl': 565, 'openssl': 620, 'zlib': 48}
 
deleting 1269 out of 6666
 
deleted: {'openssl': 649, 'curl': 592, 'zlib': 25}
 
Class Mappings:  {0: 'curl', 1: 'openssl', 2: 'zlib'}
 
Number of samples:  42
